## Not used for thesis

In [1]:
ENV["GKS_ENCODING"]="utf-8"

using LinearAlgebra
using Statistics
using Plots
using JSON
using HDF5
using StatsPlots

gr(fontfamily="Helvetica")
#default(fontfamily="Helvetica")

Plots.GRBackend()

In [2]:
colors = [:orange, :black, :gold, :blue, :red, :brown, :bisque4, :gray, :blue, :black] #惑星の色
AU = 1.495978e8 #AU

1.495978e8

In [6]:
function loadMass(key, epoch)
    file = JSON.parsefile("../Resources/nasa_data.json")
    data = file[epoch][key]
    return data["mass"]
end

loadMass (generic function with 1 method)

In [7]:
function diff_two_orbit(f1, f2, tEnd, dt, chunk_size, index1, index2)
    Nt = Int(ceil(tEnd/dt))
    dist = zeros(Nt)
    for c in 1:Int(Nt/chunk_size)
        chunk1 = read(f1, "/data/$c")
        chunk2 = read(f2, "/data/$c")
        for i in 1:chunk_size
            dist[chunk_size*(c-1)+i] = sqrt(
                (chunk1[index1,1,i] - chunk2[index2,1,i])^2 +
                (chunk1[index1,2,i] - chunk2[index2,2,i])^2 + 
                (chunk1[index1,3,i] - chunk2[index2,3,i])^2
            ) / AU
        end
    end
    return dist
end
function diff_two_orbit_bary(f1, f2, tEnd, dt, chunk_size, index1, index2, m_earth, m_moon)
    Nt = Int(ceil(tEnd/dt))
    dist = zeros(Nt)
    moon = 10
    for c in 1:Int(Nt/chunk_size)
        chunk1 = read(f1, "/data/$c")
        chunk2 = read(f2, "/data/$c")
        for i in 1:chunk_size
            barycenter = [
                chunk2[index2,1,i] * m_earth + chunk2[moon,1,i] * m_moon
                chunk2[index2,2,i] * m_earth + chunk2[moon,2,i] * m_moon
                chunk2[index2,3,i] * m_earth + chunk2[moon,3,i] * m_moon
            ] ./ (m_earth + m_moon)
            dist[chunk_size*(c-1)+i] = sqrt(
                (chunk1[index1,1,i] - barycenter[1])^2 +
                (chunk1[index1,2,i] - barycenter[2])^2 + 
                (chunk1[index1,3,i] - barycenter[3])^2
            ) / AU
        end
    end
    return dist
end

diff_two_orbit_bary (generic function with 1 method)

In [8]:
f1 = h5open("./results/dt1hr_tEnd36000d/k_all.h5", "r")
f2 = h5open("./results/dt1hr_tEnd36000d/n_earth_onl.h5", "r")
f3 = h5open("./results/dt1hr_tEnd36000d/k_all_bary.h5", "r")
f4 = h5open("./results/dt1hr_tEnd36000d/n_all.h5", "r")
tEnd = read(f1, "/params/tend")
dt = read(f1, "/params/dt")
chunk_size = read(f1, "/params/chunksize")

epoch = "2024-04-01"
m_earth = loadMass("earth", epoch)
m_moon = loadMass("moon", epoch)

@time dist1 = diff_two_orbit(f1, f2, tEnd, dt, chunk_size, 4, 2)
@time dist2 = diff_two_orbit_bary(f3, f4, tEnd, dt, chunk_size, 4, 4, m_earth, m_moon)

close(f1)
close(f2)
close(f3)
close(f4)

  0.914130 seconds (18.99 M allocations: 514.387 MiB, 61.76% gc time, 1.88% compilation time)
  1.648069 seconds (45.02 M allocations: 1.139 GiB, 54.66% gc time, 2.85% compilation time)


In [9]:
p = plot(dist1, legend=false, dpi=300, xlabel=" ", ylabel=" ", color="red")
plot!(p, dist2, legend=false, color="blue")
annotate!(p, [length(dist1)-30000, length(dist2)-30000], [dist1[end]+0.01, dist2[end]-0.02], [("$(round(dist1[end], sigdigits=3))", 12), ("$(round(dist2[end], sigdigits=3))", 12)])
savefig(p, "diff4.png")

GKS: invalid bitmap size
GKS: invalid bitmap size


"/Users/shim/Desktop/Tanq/Planet/Gravitational_Simulation/diff4.png"